# Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
#workhorses
import numpy as np
import pandas as pd

#plotters
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

#modellers
from causalimpact import CausalImpact
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from scipy import stats as spstats

#extra
import sys
from datetime import timedelta

In [5]:
#plotly settings
plt_template="plotly_dark"


# Load Data

In [8]:
df_useu=pd.read_csv('./.to_ignore/data/DEXUSEU.csv')
df_useu[:3]

,DATE,DEXUSEU
0,2012-10-09,1.2876
1,2012-10-10,1.2888
2,2012-10-11,1.2928


In [10]:
df_usuk=pd.read_csv('./.to_ignore/data/DEXUSUK.csv')
df_usuk[:3]

,DATE,DEXUSUK
0,2012-10-09,1.5987
1,2012-10-10,1.6007
2,2012-10-11,1.6028


In [7]:
df_chus=pd.read_csv('./.to_ignore/data/DEXCHUS.csv')
df_chus[:3]

,DATE,DEXCHUS
0,2012-10-09,6.2877
1,2012-10-10,6.2831
2,2012-10-11,6.2765


In [42]:
#merge all
df = pd.merge(left=df_useu, left_on=df_useu.DATE,\
              right=df_usuk, right_on=df_usuk.DATE,\
              suffixes=('_l', '_r')
             ).rename(columns={'key_0':'DATE'})\
            .dropna().drop(columns=['DATE_l','DATE_r'])

df=pd.merge(left=df, left_on=df.DATE,\
              right=df_chus, right_on=df_chus.DATE,\
              suffixes=('_l', '_r')
             ).rename(columns={'key_0':'DATE'})\
            .dropna().drop(columns=['DATE_l','DATE_r'])

df.sort_values(by='DATE',ascending=True,inplace=True)
df[:3]


,DATE,DEXUSEU,DEXUSUK,DEXCHUS
0,2012-10-09,1.2876,1.5987,6.2877
1,2012-10-10,1.2888,1.6007,6.2831
2,2012-10-11,1.2928,1.6028,6.2765


# Quick visuals of the time series


In [43]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.DATE, y=df.DEXUSEU,
                    mode='lines+markers',
                    name='usd-eur'))
fig.add_trace(go.Scatter(x=df.DATE, y=df.DEXUSUK,
                    mode='lines+markers',
                    name='usd-pound'))
#fig.add_trace(go.Scatter(x=df.DATE, y=df.DEXCHUS,
#                    mode='lines+markers', name='chy-usd'))

fig.show()

# Set constants for forecasting

In [9]:
# Set forecasting constants (no need to change these)

# Training data prior to event
PRE_PERIOD=[pd.to_datetime('2012-01-03'), pd.to_datetime('2016-06-23')]

# Duration of period after the event to analyze
POST_PERIOD_LENGTH=28
POST_PERIOD=[pd.to_datetime('2016-06-24'), pd.to_datetime('2016-07-22')]

# Lookback window used for forecasting (usually 1-5x the forecast horizon)
CONTEXT_WINDOW = 1 * POST_PERIOD_LENGTH